In [1]:
import xlwings as xw
import pandas as pd
import json

In [2]:
wb = xw.Book('Matrix.xlsx')
#wb = xw.Book(r'C:\Users\Porter\OneDrive - Danmarks Tekniske Universitet\Christian Bertram\MDM.xlsx')

In [3]:
totalmdm = wb.sheets['Sheet1']

In [4]:
mdmdata = totalmdm.range('A1:BL65').value

In [5]:
#get nodes
mdmnodes = mdmdata[2] #first row
for node in mdmnodes:
    if not node:
        print node

None
None
None


In [6]:
#assign levels to nodes
newnodes = []
for index, mdmnode in enumerate(mdmnodes):
    node = {}
    if (mdmnode != 'None') and (index != 0) and mdmnode:
        #print mdmnode
        if isinstance(mdmnode, float):
            mdmnode = str(int(mdmnode))
        node["name"] = mdmnode.strip() #not tested
        node["level"] = mdmdata[1][index]
        node["index"] = index - 3
        node["children"] = []
        #print mdmnode, node["level"]
        newnodes.append(node)

In [7]:
#map node names to indices
nodesdict = {}
for node in newnodes:
    nodesdict[node["name"]] = node["index"]
nodesdict

{u'Combustion Control': 35,
 u'Combustion Process': 43,
 u'Comfort functions': 10,
 u'F1-1': 11,
 u'F1-10': 20,
 u'F1-2': 12,
 u'F1-3': 13,
 u'F1-4': 14,
 u'F1-5': 15,
 u'F1-6': 16,
 u'F1-7': 17,
 u'F1-8': 18,
 u'F1-9': 19,
 u'F2-1': 22,
 u'F2-10': 31,
 u'F2-11': 32,
 u'F2-12': 33,
 u'F2-13': 34,
 u'F2-2': 23,
 u'F2-3': 24,
 u'F2-4': 25,
 u'F2-5': 26,
 u'F2-6': 27,
 u'F2-7': 28,
 u'F2-8': 29,
 u'F2-9': 30,
 u'F3-1': 36,
 u'F3-2': 37,
 u'F3-3': 38,
 u'F3-4': 39,
 u'F3-5': 40,
 u'F3-6': 41,
 u'F3-7': 42,
 u'F4-1': 44,
 u'F4-2': 45,
 u'F4-3': 46,
 u'F4-4': 47,
 u'F4-5': 48,
 u'F4-6': 49,
 u'F4-7': 50,
 u'F4-8': 51,
 u'F4-9': 52,
 u'FUNCTIONS': 9,
 u'Functional Interface': 53,
 u'I-1': 54,
 u'I-2': 55,
 u'I-3': 56,
 u'I-4': 57,
 u'I-5': 58,
 u'I-6': 59,
 u'I-7': 60,
 u'Partymode': 21,
 u'R1': 1,
 u'R2': 2,
 u'R3': 3,
 u'R4': 4,
 u'R5': 5,
 u'R6': 6,
 u'R7': 7,
 u'R8': 8,
 u'REQUIREMENTS': 0}

In [8]:
def attachChildren(startindex):
    print "attachChildren of node: ", newnodes[startindex]['name'] 
    #startindex = node.index
    startnode = newnodes[startindex]
    children = findChildren(startindex)
    print "returned children are: ", children
    startnode["children"].append(children)
    for child in children:
        attachChildren(nodesdict[child])
    return

In [9]:
#find immediate children
#iterate through next nodes / add to resultarray if node.level is next to startnode level
#if node.level equals start node level, we exit the function
def findChildren(startindex):
    resultarray = []
    print "Returning children for node ", newnodes[startindex] 
    startlevel = newnodes[startindex]["level"]
    for node in newnodes[startindex+1:]:
        print "checking node %s with index %d and level %d and startindex %d" % (node["name"], node["index"], node["level"], startindex)  
        if node["level"] == startlevel + 1:
            resultarray.append(node["name"])
            print node['name']
        elif node["level"] == startlevel:
            print "samelevel reached"
            return resultarray
    return resultarray

In [10]:
#find top level nodes
lvl1nodes = []
for node in newnodes:
    if node["level"] == 1:
        lvl1nodes.append(node["name"])
lvl1nodes

[u'REQUIREMENTS', u'FUNCTIONS', u'Functional Interface']

In [11]:
#attachChildren(0)
#for nodes that have level one, run attachChildren
for lvl1node in lvl1nodes:
    attachChildren(nodesdict[lvl1node])

attachChildren of node:  REQUIREMENTS
Returning children for node  {'index': 0, 'children': [], 'name': u'REQUIREMENTS', 'level': 1.0}
checking node R1 with index 1 and level 2 and startindex 0
R1
checking node R2 with index 2 and level 2 and startindex 0
R2
checking node R3 with index 3 and level 2 and startindex 0
R3
checking node R4 with index 4 and level 2 and startindex 0
R4
checking node R5 with index 5 and level 2 and startindex 0
R5
checking node R6 with index 6 and level 2 and startindex 0
R6
checking node R7 with index 7 and level 2 and startindex 0
R7
checking node R8 with index 8 and level 2 and startindex 0
R8
checking node FUNCTIONS with index 9 and level 1 and startindex 0
samelevel reached
returned children are:  [u'R1', u'R2', u'R3', u'R4', u'R5', u'R6', u'R7', u'R8']
attachChildren of node:  R1
Returning children for node  {'index': 1, 'children': [], 'name': u'R1', 'level': 2.0}
checking node R2 with index 2 and level 2 and startindex 1
samelevel reached
returned chi

In [12]:
#assign domains based on the lvl1node
for index, node in enumerate(newnodes):
    if node["level"] == 1:
        node["domain"] = node['name']
        otherindex = index + 1
        print index
        while newnodes[otherindex]["level"] != 1:
            newnodes[otherindex]["domain"] = node["domain"]
            print newnodes[otherindex]
            otherindex = otherindex + 1
            if otherindex == len(newnodes):
                break;

0
{'index': 1, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R1', 'level': 2.0}
{'index': 2, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R2', 'level': 2.0}
{'index': 3, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R3', 'level': 2.0}
{'index': 4, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R4', 'level': 2.0}
{'index': 5, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R5', 'level': 2.0}
{'index': 6, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R6', 'level': 2.0}
{'index': 7, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R7', 'level': 2.0}
{'index': 8, 'domain': u'REQUIREMENTS', 'children': [[]], 'name': u'R8', 'level': 2.0}
9
{'index': 10, 'domain': u'FUNCTIONS', 'children': [[u'F1-1', u'F1-2', u'F1-3', u'F1-4', u'F1-5', u'F1-6', u'F1-7', u'F1-8', u'F1-9', u'F1-10']], 'name': u'Comfort functions', 'level': 2.0}
{'index': 11, 'domain': u'FUNCTIONS', 'children': [[]], 'name': u'F1-1', 'level': 3.0}
{'index': 12, 'domain': 

In [13]:
newnodes

[{'children': [[u'R1', u'R2', u'R3', u'R4', u'R5', u'R6', u'R7', u'R8']],
  'domain': u'REQUIREMENTS',
  'index': 0,
  'level': 1.0,
  'name': u'REQUIREMENTS'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 1,
  'level': 2.0,
  'name': u'R1'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 2,
  'level': 2.0,
  'name': u'R2'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 3,
  'level': 2.0,
  'name': u'R3'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 4,
  'level': 2.0,
  'name': u'R4'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 5,
  'level': 2.0,
  'name': u'R5'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 6,
  'level': 2.0,
  'name': u'R6'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 7,
  'level': 2.0,
  'name': u'R7'},
 {'children': [[]],
  'domain': u'REQUIREMENTS',
  'index': 8,
  'level': 2.0,
  'name': u'R8'},
 {'children': [[u'Comfort functions',
    u'Partymode',
    u'C

In [14]:
#prepare string for export to JSON
jsonstr="{  \"nodes\": ["

In [15]:
#print nodes
nodesstr=''
for node in newnodes:
    print "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
    nodesstr = nodesstr + "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
nodesstr

{"name": "REQUIREMENTS","level": 1,"domain": "requirements"},
{"name": "R1","level": 2,"domain": "requirements"},
{"name": "R2","level": 2,"domain": "requirements"},
{"name": "R3","level": 2,"domain": "requirements"},
{"name": "R4","level": 2,"domain": "requirements"},
{"name": "R5","level": 2,"domain": "requirements"},
{"name": "R6","level": 2,"domain": "requirements"},
{"name": "R7","level": 2,"domain": "requirements"},
{"name": "R8","level": 2,"domain": "requirements"},
{"name": "FUNCTIONS","level": 1,"domain": "functions"},
{"name": "Comfort functions","level": 2,"domain": "functions"},
{"name": "F1-1","level": 3,"domain": "functions"},
{"name": "F1-2","level": 3,"domain": "functions"},
{"name": "F1-3","level": 3,"domain": "functions"},
{"name": "F1-4","level": 3,"domain": "functions"},
{"name": "F1-5","level": 3,"domain": "functions"},
{"name": "F1-6","level": 3,"domain": "functions"},
{"name": "F1-7","level": 3,"domain": "functions"},
{"name": "F1-8","level": 3,"domain": "functio

u'{"name": "REQUIREMENTS","level": 1,"domain": "requirements"},{"name": "R1","level": 2,"domain": "requirements"},{"name": "R2","level": 2,"domain": "requirements"},{"name": "R3","level": 2,"domain": "requirements"},{"name": "R4","level": 2,"domain": "requirements"},{"name": "R5","level": 2,"domain": "requirements"},{"name": "R6","level": 2,"domain": "requirements"},{"name": "R7","level": 2,"domain": "requirements"},{"name": "R8","level": 2,"domain": "requirements"},{"name": "FUNCTIONS","level": 1,"domain": "functions"},{"name": "Comfort functions","level": 2,"domain": "functions"},{"name": "F1-1","level": 3,"domain": "functions"},{"name": "F1-2","level": 3,"domain": "functions"},{"name": "F1-3","level": 3,"domain": "functions"},{"name": "F1-4","level": 3,"domain": "functions"},{"name": "F1-5","level": 3,"domain": "functions"},{"name": "F1-6","level": 3,"domain": "functions"},{"name": "F1-7","level": 3,"domain": "functions"},{"name": "F1-8","level": 3,"domain": "functions"},{"name": "F

In [16]:
jsonstr = jsonstr + nodesstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"links\": ["
jsonstr

u'{  "nodes": [{"name": "REQUIREMENTS","level": 1,"domain": "requirements"},{"name": "R1","level": 2,"domain": "requirements"},{"name": "R2","level": 2,"domain": "requirements"},{"name": "R3","level": 2,"domain": "requirements"},{"name": "R4","level": 2,"domain": "requirements"},{"name": "R5","level": 2,"domain": "requirements"},{"name": "R6","level": 2,"domain": "requirements"},{"name": "R7","level": 2,"domain": "requirements"},{"name": "R8","level": 2,"domain": "requirements"},{"name": "FUNCTIONS","level": 1,"domain": "functions"},{"name": "Comfort functions","level": 2,"domain": "functions"},{"name": "F1-1","level": 3,"domain": "functions"},{"name": "F1-2","level": 3,"domain": "functions"},{"name": "F1-3","level": 3,"domain": "functions"},{"name": "F1-4","level": 3,"domain": "functions"},{"name": "F1-5","level": 3,"domain": "functions"},{"name": "F1-6","level": 3,"domain": "functions"},{"name": "F1-7","level": 3,"domain": "functions"},{"name": "F1-8","level": 3,"domain": "functions"

In [17]:
#generate links
links = []
for node in newnodes:
    #print { "source": "Day and night", "target": "Priority A"  },
    print node["name"]
    print node["children"]
    for child in node["children"][0]:
        #print child
        link = {}
        link["source"] = str(node["name"])
        link["target"] = str(child)
        links.append(link)
        #print link
linkstr = ''
for link in links:
    linkstr = linkstr + "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  
    print "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  


REQUIREMENTS
[[u'R1', u'R2', u'R3', u'R4', u'R5', u'R6', u'R7', u'R8']]
R1
[[]]
R2
[[]]
R3
[[]]
R4
[[]]
R5
[[]]
R6
[[]]
R7
[[]]
R8
[[]]
FUNCTIONS
[[u'Comfort functions', u'Partymode', u'Combustion Control', u'Combustion Process']]
Comfort functions
[[u'F1-1', u'F1-2', u'F1-3', u'F1-4', u'F1-5', u'F1-6', u'F1-7', u'F1-8', u'F1-9', u'F1-10']]
F1-1
[[]]
F1-2
[[]]
F1-3
[[]]
F1-4
[[]]
F1-5
[[]]
F1-6
[[]]
F1-7
[[]]
F1-8
[[]]
F1-9
[[]]
F1-10
[[]]
Partymode
[[u'F2-1', u'F2-2', u'F2-3', u'F2-4', u'F2-5', u'F2-6', u'F2-7', u'F2-8', u'F2-9', u'F2-10', u'F2-11', u'F2-12', u'F2-13']]
F2-1
[[]]
F2-2
[[]]
F2-3
[[]]
F2-4
[[]]
F2-5
[[]]
F2-6
[[]]
F2-7
[[]]
F2-8
[[]]
F2-9
[[]]
F2-10
[[]]
F2-11
[[]]
F2-12
[[]]
F2-13
[[]]
Combustion Control
[[u'F3-1', u'F3-2', u'F3-3', u'F3-4', u'F3-5', u'F3-6', u'F3-7']]
F3-1
[[]]
F3-2
[[]]
F3-3
[[]]
F3-4
[[]]
F3-5
[[]]
F3-6
[[]]
F3-7
[[]]
Combustion Process
[[u'F4-1', u'F4-2', u'F4-3', u'F4-4', u'F4-5', u'F4-6', u'F4-7', u'F4-8', u'F4-9']]
F4-1
[[]]
F4-2
[[]]
F4-3
[[]]


In [18]:
jsonstr = jsonstr + linkstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"traces\": ["
jsonstr

u'{  "nodes": [{"name": "REQUIREMENTS","level": 1,"domain": "requirements"},{"name": "R1","level": 2,"domain": "requirements"},{"name": "R2","level": 2,"domain": "requirements"},{"name": "R3","level": 2,"domain": "requirements"},{"name": "R4","level": 2,"domain": "requirements"},{"name": "R5","level": 2,"domain": "requirements"},{"name": "R6","level": 2,"domain": "requirements"},{"name": "R7","level": 2,"domain": "requirements"},{"name": "R8","level": 2,"domain": "requirements"},{"name": "FUNCTIONS","level": 1,"domain": "functions"},{"name": "Comfort functions","level": 2,"domain": "functions"},{"name": "F1-1","level": 3,"domain": "functions"},{"name": "F1-2","level": 3,"domain": "functions"},{"name": "F1-3","level": 3,"domain": "functions"},{"name": "F1-4","level": 3,"domain": "functions"},{"name": "F1-5","level": 3,"domain": "functions"},{"name": "F1-6","level": 3,"domain": "functions"},{"name": "F1-7","level": 3,"domain": "functions"},{"name": "F1-8","level": 3,"domain": "functions"

In [19]:
#print traces
tracestr = ''
for rowindex, row in enumerate(mdmdata):
    #print row
    for colindex, cell in enumerate(row):
        #if (cell > 0) and (cell != 'None') and (rowindex > 1) and (colindex > 1) and (rowindex != colindex):
        if (cell > 0) and (cell != 'None') and (rowindex > 3) and (colindex > 3) and (rowindex != colindex):
            #print cell, rowindex-2, colindex-1
            tracestr = tracestr + "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            print "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            #print newnodes[rowindex-4]["name"], newnodes[colindex-3]["name"] 

{ "nodeids":"1,24" },
{ "nodeids":"1,25" },
{ "nodeids":"1,32" },
{ "nodeids":"2,24" },
{ "nodeids":"2,25" },
{ "nodeids":"2,32" },
{ "nodeids":"3,26" },
{ "nodeids":"3,27" },
{ "nodeids":"3,29" },
{ "nodeids":"3,30" },
{ "nodeids":"4,28" },
{ "nodeids":"5,22" },
{ "nodeids":"5,34" },
{ "nodeids":"6,23" },
{ "nodeids":"6,27" },
{ "nodeids":"6,31" },
{ "nodeids":"6,33" },
{ "nodeids":"7,24" },
{ "nodeids":"7,28" },
{ "nodeids":"8,22" },
{ "nodeids":"8,25" },
{ "nodeids":"8,29" },
{ "nodeids":"12,18" },
{ "nodeids":"13,16" },
{ "nodeids":"14,11" },
{ "nodeids":"15,13" },
{ "nodeids":"16,57" },
{ "nodeids":"17,12" },
{ "nodeids":"17,31" },
{ "nodeids":"18,17" },
{ "nodeids":"18,55" },
{ "nodeids":"19,56" },
{ "nodeids":"20,18" },
{ "nodeids":"20,50" },
{ "nodeids":"22,28" },
{ "nodeids":"22,50" },
{ "nodeids":"25,22" },
{ "nodeids":"25,33" },
{ "nodeids":"26,30" },
{ "nodeids":"26,55" },
{ "nodeids":"27,31" },
{ "nodeids":"27,57" },
{ "nodeids":"28,24" },
{ "nodeids":"29,26" },
{ "nodeids

In [20]:
jsonstr = jsonstr + tracestr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "] }"
jsonstr

u'{  "nodes": [{"name": "REQUIREMENTS","level": 1,"domain": "requirements"},{"name": "R1","level": 2,"domain": "requirements"},{"name": "R2","level": 2,"domain": "requirements"},{"name": "R3","level": 2,"domain": "requirements"},{"name": "R4","level": 2,"domain": "requirements"},{"name": "R5","level": 2,"domain": "requirements"},{"name": "R6","level": 2,"domain": "requirements"},{"name": "R7","level": 2,"domain": "requirements"},{"name": "R8","level": 2,"domain": "requirements"},{"name": "FUNCTIONS","level": 1,"domain": "functions"},{"name": "Comfort functions","level": 2,"domain": "functions"},{"name": "F1-1","level": 3,"domain": "functions"},{"name": "F1-2","level": 3,"domain": "functions"},{"name": "F1-3","level": 3,"domain": "functions"},{"name": "F1-4","level": 3,"domain": "functions"},{"name": "F1-5","level": 3,"domain": "functions"},{"name": "F1-6","level": 3,"domain": "functions"},{"name": "F1-7","level": 3,"domain": "functions"},{"name": "F1-8","level": 3,"domain": "functions"

In [21]:
with open('graph_nov21_from_matrix.json', 'w') as outfile:  
    outfile.write(jsonstr)

## One domain

In [ ]:
sht = wb.sheets['Nodes']

In [87]:
nodestwo = sht.range('A1:C20').value
nodestwo

[[u'A0', u'Information', 1.0],
 [u'A1', u'Business', 2.0],
 [u'A2', u'- Client', 3.0],
 [u'A3', u'- Requirements', 3.0],
 [u'A4', u'Configuration', 2.0],
 [u'A5', u'- Plant Type', 3.0],
 [u'A6', u'- Plant Size', 3.0],
 [u'A7', u'- Dimensions', 3.0],
 [u'A8', u'- Intended product/powder', 3.0],
 [u'A9', u'- Interface with building', 3.0],
 [u'A10', u'- BOM', 3.0],
 [u'A11', u'Operation', 2.0],
 [u'A12', u'- Process Parameters', 3.0],
 [u'A13', u'- Realized processdata', 3.0],
 [u'A14', u'Project', 2.0],
 [u'A15', u'- Planning and Scheduling', 3.0],
 [u'A16', u'- Resources (e.g. hours)', 3.0],
 [u'A17', u'- Pricing', 3.0],
 [u'A18', u'- Procurement', 3.0],
 [u'A19', u'- Cost', 3.0]]

In [89]:
for node in nodestwo:
    print "{\"name\": \"" + node[1].strip() + "\",\"level\": " + str(int(round(node[2]))).strip() + ",\"domain\": \"usecases\"}," 

{"name": "Information","level": 1,"domain": "usecases"},
{"name": "Business","level": 2,"domain": "usecases"},
{"name": "- Client","level": 3,"domain": "usecases"},
{"name": "- Requirements","level": 3,"domain": "usecases"},
{"name": "Configuration","level": 2,"domain": "usecases"},
{"name": "- Plant Type","level": 3,"domain": "usecases"},
{"name": "- Plant Size","level": 3,"domain": "usecases"},
{"name": "- Dimensions","level": 3,"domain": "usecases"},
{"name": "- Intended product/powder","level": 3,"domain": "usecases"},
{"name": "- Interface with building","level": 3,"domain": "usecases"},
{"name": "- BOM","level": 3,"domain": "usecases"},
{"name": "Operation","level": 2,"domain": "usecases"},
{"name": "- Process Parameters","level": 3,"domain": "usecases"},
{"name": "- Realized processdata","level": 3,"domain": "usecases"},
{"name": "Project","level": 2,"domain": "usecases"},
{"name": "- Planning and Scheduling","level": 3,"domain": "usecases"},
{"name": "- Resources (e.g. hours)",

In [ ]:
links = {}
source = 

In [92]:
nodesdict = {}
for index, node in enumerate(nodestwo):
    nodesdict[node[0]] = index
nodesdict

{u'A0': 0,
 u'A1': 1,
 u'A10': 10,
 u'A11': 11,
 u'A12': 12,
 u'A13': 13,
 u'A14': 14,
 u'A15': 15,
 u'A16': 16,
 u'A17': 17,
 u'A18': 18,
 u'A19': 19,
 u'A2': 2,
 u'A3': 3,
 u'A4': 4,
 u'A5': 5,
 u'A6': 6,
 u'A7': 7,
 u'A8': 8,
 u'A9': 9}

In [93]:
celllist = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#for index,topcells in enumerate(columns, start = 1):
for index in range(7,20):
    cellstr = celllist[index].upper() + "2:" + celllist[index].upper() + "20"
    source = celllist[index].upper() + "1"
    #print cellstr
    sourcecol = sht.range(cellstr).value
    #print sourcecol
    for cell in sourcecol:
        if (cell != -2146826259) and (sht.range(source).value != cell):
            #print sht.range(source).value, cell
            #print nodesdict[sht.range(source).value], nodesdict[cell]
            print "{ \"nodeids\":\"" + str(nodesdict[sht.range(source).value]) + "," + str(nodesdict[cell]) + "\" },"

{ "nodeids":"1,2" },
{ "nodeids":"1,3" },
{ "nodeids":"1,15" },
{ "nodeids":"2,3" },
{ "nodeids":"2,15" },
{ "nodeids":"3,4" },
{ "nodeids":"3,5" },
{ "nodeids":"3,6" },
{ "nodeids":"3,7" },
{ "nodeids":"3,10" },
{ "nodeids":"3,11" },
{ "nodeids":"3,12" },
{ "nodeids":"3,13" },
{ "nodeids":"3,15" },
{ "nodeids":"3,18" },
{ "nodeids":"3,19" },
{ "nodeids":"4,5" },
{ "nodeids":"4,6" },
{ "nodeids":"4,7" },
{ "nodeids":"4,9" },
{ "nodeids":"4,10" },
{ "nodeids":"4,15" },
{ "nodeids":"4,18" },
{ "nodeids":"4,19" },
{ "nodeids":"5,6" },
{ "nodeids":"5,7" },
{ "nodeids":"5,10" },
{ "nodeids":"5,15" },
{ "nodeids":"5,16" },
{ "nodeids":"5,17" },
{ "nodeids":"5,18" },
{ "nodeids":"5,19" },
{ "nodeids":"6,7" },
{ "nodeids":"6,10" },
{ "nodeids":"6,15" },
{ "nodeids":"6,16" },
{ "nodeids":"6,17" },
{ "nodeids":"6,18" },
{ "nodeids":"6,19" },
{ "nodeids":"7,10" },
{ "nodeids":"7,18" },
{ "nodeids":"7,19" },
{ "nodeids":"8,16" },
{ "nodeids":"8,18" },
{ "nodeids":"8,19" },
{ "nodeids":"9,18" },
{ 

In [20]:
for cell in column:
    if source != cell:
        print source, cell

A1 A2
A1 A3
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 A15
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
